# FEgrow - congeneric ligands from a template

### Developers: Mat Bieniek, Ben Cree, Rachael Pirie, Josh Horton

* Add R-groups in user-defined positions
* Output ADMET properties
* Perform constrained optimisation
* Score poses
* Send output to free energy calculations

Open source, customisable, fast/easy to use.

## Overview

This notebook demonstrates the entire rgroups workflow for generating a series of ligands with a common core for a specific binding site, by the addition of a user-defined set of R-groups. 

These de novo ligands are then subjected to ADMET analysis. Valid conformers are enumerated, and optimised in the receptor using hybrid machine learning / molecular mechanics potentials (ML/MM). 

An ensemble of low energy conformers is generated for each ligand, and scored using a convolutional neural network (CNN), then finally compared to experimental data (where available).

The target for this tutorial is PTP1B. [1][2]

In [ ]:
import copy

import prody
from rdkit import Chem

import fegrow
from fegrow import RGroups

# Prepare the ligand template

Protonate the template structure with open babel

In [ ]:
!obabel sarscov2/lig.pdb -O sarscov2/coreh.sdf -p 7

Load the protonated ligand into FEgrow

In [ ]:
init_mol = Chem.SDMolSupplier('sarscov2/coreh.sdf', removeHs=False)[0]

# get the FEgrow representation of the rdkit Mol
template_mol = fegrow.RMol(init_mol)

Show the 2D (with indices) representation of the core. This is used to select the desired growth vector.

In [ ]:
template_mol.rep2D(idx=True, size=(500, 500))

Using the 2D drawing, select a hydrogen index for the R-group

In [ ]:
attachment_index = [40]

R-groups can be selected interactively or programmaticaly. Tailor .mol files can be used as well as long as their hydrogen is replaced with the *element* R. 

In [ ]:
# interactive selection
RGroups

In [ ]:
# retrieve the interactively selected groups
interactive_rgroups = RGroups.get_selected()

# you can also directly access the built-in dataframe programmatically
groups = RGroups.dataframe
R_group_ethanol = groups.loc[groups['Name']=='ethanol']['Mol'].values[0]
R_group_cyclopropane = groups.loc[groups['Name'] == 'cyclopropane' ]['Mol'].values[0]

# add your own R-groups files
R_group_methanol = Chem.MolFromMolFile('manual_rgroups/methanol-r.mol', removeHs=False)

# make a list of R-group molecule
selected_rgroups = [R_group_methanol, R_group_ethanol, R_group_cyclopropane] + interactive_rgroups
selected_rgroups

# Build a congeneric series

Merge the template with the R-groups

In [ ]:
rmols = fegrow.build_molecules(template_mol, 
                               attachment_index, 
                               selected_rgroups)

In [ ]:
rmols.rep2D()

In [ ]:
rmols[0].rep3D()

Once the ligands have been generated, they can be assessed for various ADMET properties, including Lipinksi rule of 5 properties, the presence of unwanted substructures or problematic functional groups, and synthetic accessibility. [3][4]

In [ ]:
rmols.toxicity()

For each ligand, a specified number of conformers (`num_conf`) is generated by using the RDKit ETKDG algorithm.[5] Conformers that are too similar to the initial structure are discarded. Empirically, we have found that `num_conf=200` gives an exhaustive search, and `num_conf=50` gives a reasonable, fast search, in most cases.

If required, a third argument can be added `flexible=[0,1,...]`, which provides a list of additional atoms in the core that can be allowed to be flexible. This is useful, for example, if growing from a methyl group and you would like the added R-group to freely rotate.

In [ ]:
rmols.generate_conformers(num_conf=10, 
                         minimum_conf_rms=0.5, 
                        )
                        # flexible=[39, 18, 40, 16, 17, 19, 5, 6, 36, 2, 1, 33, 3, 4, 35, 34, 15, 38])

In [ ]:
#rmol.rep3D(template=True)

### Prepare the protein

The protein structure initially is not protonated, this along with other repairs is done using PDBFixer [6]. 

In [ ]:
# load the complex with the ligand
sys = prody.parsePDB('sarscov2/7l10.pdb')

# remove any unwanted molecules
rec = sys.select('not (nucleic or hetatm or water)')

# save the processed protein
prody.writePDB('rec.pdb', rec)

# fix the receptor file (missing residues, protonation, etc)
fegrow.fix_receptor("rec.pdb", "rec_final.pdb")

# load back into prody
rec_final = prody.parsePDB("rec_final.pdb")

In [ ]:
rmols[0].rep3D(prody=rec_final)

Any conformers that clash with the protein (within 1 A of the backbone), are removed.

In [ ]:
rmols.remove_clashing_confs(rec_final)

In [ ]:
rmols[0].rep3D(prody=rec_final)

### Optimise conformers in context of protein

The remaining conformers are optimised using hybrid machine learning / molecular mechanics (ML/MM), using the ANI [cite]neural nework potential for the ligand energetics (as long as it contains only the atoms H, C, N, O, F, S, Cl). (Note that the Open Force Field Parsley force field is used otherwise).

In [ ]:
# opt_mol, energies
energies = rmols.optimise_in_receptor(
    receptor_file="rec_final.pdb", 
    ligand_force_field="openff", 
    use_ani=False,
    sigma_scale_factor=0.8,
    relative_permittivity=4,
    water_model = None
)

In [ ]:
print(energies)

Any of the rmols that have no conformers (due to clashing with the protein) can be discarded using the .discard_missing() function. This function also returns a list of the indexes that were removed, which will be used in order to carry out data analysis.

In [ ]:
missing_ids = rmols.discard_missing()

In [ ]:
rmols[0].rep3D()

In [ ]:
rmols[0].to_file("optimised_coords.pdb")

Conformers are now sorted by energy, only retaining those within 5 kcal/mol of the lowest energy structure.

In [ ]:
final_energies = rmols.sort_conformers(energy_range=5)

Save all of the best conformers to files

In [ ]:
[rmol.to_file(f"best_conformers{i}.pdb") for i, rmol in enumerate(rmols)]

In [ ]:
print(final_energies)

The conformers are scored by using Gnina [7] on each conformer of each ligand, which utilises a convolutional neural network. Gnina is downloaded during the first time it is used. MAC is currently not supported. 

In [ ]:
CNNscores, ic50s = rmols.gnina(receptor_file="rec_final.pdb") 
CNNscores

IC50 values are also predicted from the CNNscores

In [ ]:
ic50s

The gnina score can be converted to predicted IC50.

References:

    1 - https://pubs.acs.org/doi/10.1021/ja512751q
    
    2 - https://pubs.acs.org/doi/10.1021/jm0702478
    
    3 - https://www.sciencedirect.com/science/article/pii/S0169409X96004231
    
    4 - http://www.jcheminf.com/content/1/1/8
    
    5 - https://pubs.acs.org/doi/pdf/10.1021/acs.jcim.5b00654
    
    6 - https://academic.oup.com/bioinformatics/article/37/20/3657/6211036
    
    7 - https://github.com/openmm/pdbfixer